## 什么是数据连接？

LLM应用往往需要用户特定的数据，而这些数据并不属于模型的训练集。`LangChain` 的数据连接概念，
通过提供以下组件，实现用户数据的加载、转换、存储和查询：

- 文档加载器：从不同的数据源加载文档
- 文档转换器：拆分文档，将文档转换为问答格式，去除a冗余文档，等等
- 文本嵌入模型：将非结构化文本转换为浮点数数组表现形式，也称为向量
- 向量存储：存储和搜索嵌入数据（向量）
- 检索器：提供数据查询的通用接口

我们通过下一段落的实践，来介绍这些组件的使用。
## 数据连接实践

在LLM应用连接用户数据时，通常我们会以如下步骤完成：
1. 加载文档
2. 拆分文档
3. 向量化文档分块
4. 向量数据存储

这样，我们就可以通过向量数据的检索器，来查询用户数据。接下来我们看看每一步的代码实现示例。
最后，我们将通过一个完整的示例来演示如何使用数据连接。

### 加载文档

`Langchain` 提供了多种文档加载器，用于从不同的数据源加载不同类型的文档。
比如，我们可以从本地文件系统加载文档，也可以通过网络加载远程数据。想了解 `Langchain` 所支持的所有文档加载器，
请参考[Document Loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)。

### 环境配置
我们需要为此安装所需的软件包：

```CMD
conda install faiss-cpu
```

安装 package 的详细版本信息
faiss-1.7.4
faiss-cpu-1.7.4
Libfaiss-1.7.4
libfaiss-avx2-1.7.4

在本节课程中，我们将使用最基本的 `TextLoader` 来加载本地文件系统中的文档。代码如下：


In [1]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# 如果你需要通过代理端口访问，你需要如下配置
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

In [2]:
# 1.加载文档
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./LongText.md", encoding="utf-8")
docs = loader.load()
docs

[Document(page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain\n\n最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**\n\n根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。\n\n推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介\n\n大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠LLM还不足以创建一个真正强大的应用程序。它还需要其他计算资源或知识来源。\n\n`Langchain` 旨在帮助开发这些类型应用程序，比如：\n- 基于文档数据的问答\n- 聊天机器人\n- 代理\n\n## OpenAI 简介\n\n`OpenAI` 是LLM生态的模型层最大的玩家之一。大家目前熟知的 *GPT-3.5*，*GPT-4* 等模型都是OpenAI的产品。它的API允许开发人员通过简单的API调用来访问这些模型。\n\n## Langchain与OpenAI\n\n`Lang

In [7]:
# 2.拆分文档
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
)

split_docs = text_splitter.split_documents(docs)
print(len(docs[0].page_content))
print(len(split_docs))

sum = 0
for split_doc in split_docs:
    cal = len(split_doc.page_content)
    print(cal)
    sum += cal
print("sum = ", sum)

996
9
459
470
63
296
451
468
48
495
375
sum =  3125


In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(split_docs, embeddings)

In [20]:
query = "什么是WTF Langchain？"
sim_docs = db.similarity_search_with_score(query, 3)
print(sim_docs[0][0].page_content)

---
title: 01. Hello Langchain
tags:
  - openai
  - llm
  - langchain
---

# WTF Langchain极简入门: 01. Hello Langchain

最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：
- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)
- LLM使用OpenAI的模型
- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**

根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。


In [21]:
sim_docs

[(Document(page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain\n\n最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**\n\n根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。', metadata={'source': './LongText.md'}),
  0.28679264),
 (Document(page_content='推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介\n\n大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠LLM还不足以创建一个真正强大的应用程序。它还需要其他计算资源或知识来源。\n\n`Langchain` 旨在帮助开发这些类型应用程序，比如：\n- 基于文档数据的问答\n- 聊天机器人\n- 代理\n\n## OpenAI 简介\n\n`OpenAI` 是LLM生态的模型层最大的玩家之一。大家目前熟知的 *GPT-3.5*，*GPT-

In [22]:
for doc, score in sim_docs:
    print(f"Document: {doc}, Similarity Score: {score}")

Document: page_content='---\ntitle: 01. Hello Langchain\ntags:\n  - openai\n  - llm\n  - langchain\n---\n\n# WTF Langchain极简入门: 01. Hello Langchain\n\n最近在学习Langchain框架，顺手写一个“WTF Langchain极简入门”，供小白们使用（编程大佬可以另找教程）。本教程默认以下前提：\n- 使用Python版本的[Langchain](https://github.com/hwchase17/langchain)\n- LLM使用OpenAI的模型\n- Langchain目前还处于快速发展阶段，版本迭代频繁，为避免示例代码失效，本教程统一使用版本 **0.0.235**\n\n根据Langchain的[代码约定](https://github.com/hwchase17/langchain/blob/v0.0.235/pyproject.toml#L14C1-L14C24)，Python版本 ">=3.8.1,<4.0"。' metadata={'source': './LongText.md'}, Similarity Score: 0.2867926359176636
Document: page_content='推特：[@verysmallwoods](https://twitter.com/verysmallwoods)\n\n所有代码和教程开源在github: [github.com/sugarforever/wtf-langchain](https://github.com/sugarforever/wtf-langchain)\n\n-----\n\n## Langchain 简介\n\n大型语言模型（LLM）正在成为一种具有变革性的技术，使开发人员能够构建以前无法实现的应用程序。然而，仅仅依靠LLM还不足以创建一个真正强大的应用程序。它还需要其他计算资源或知识来源。\n\n`Langchain` 旨在帮助开发这些类型应用程序，比如：\n- 基于文档数据的问答\n- 聊天机器人\n- 代理\n\n## OpenAI 简介\n\n`OpenAI` 是LLM生态的模型层最大的玩家之一。大家目前熟

In [10]:
print(db)

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "什么是 WTF Langchain？"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

WTF Langchain 是一个极简入门教程，用于帮助初学者学习 Langchain 框架。它默认使用 Python 版本的 Langchain，LLM 使用 OpenAI 的模型。教程中提供了安装指令和示例代码，以帮助用户快速了解 Langchain 和 OpenAI 的关系，并创建第一个 Langchain 应用程序。


使用 chromabd

```cmd
conda install chromadb
```

安装 package 的详细版本信息
onnxruntime-1.17.1